# Flask Example Ungraded Lab

In [4]:
!pip install Flask-WTF

In [18]:
from flask import Flask, request, jsonify, abort
from flask_sqlalchemy import SQLAlchemy
from flask_wtf.csrf import CSRFProtect
from marshmallow import Schema, fields, validate
from werkzeug.security import generate_password_hash, check_password_hash
import threading
import logging

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'your_secret_key'  # Use a strong, random secret key
csrf = CSRFProtect(app)

db = SQLAlchemy(app)

# Configure logging
logging.basicConfig(level=logging.INFO)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class UserSchema(Schema):
    id = fields.Int(dump_only=True)
    username = fields.Str(required=True, validate=validate.Length(min=1, max=80))
    password = fields.Str(required=True, validate=validate.Length(min=6))

user_schema = UserSchema()
users_schema = UserSchema(many=True)

with app.app_context():
    db.create_all()

@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

@app.route('/users', methods=['GET'])
def get_users():
    try:
        users = User.query.all()
        result = users_schema.dump(users)
        return jsonify(result)
    except Exception as e:
        app.logger.error("Error fetching users: %s", str(e))
        return jsonify({"message": "An error occurred while fetching users"}), 500

@app.route('/user/<int:id>', methods=['GET'])
def get_user(id):
    try:
        user = User.query.get(id)
        if user:
            result = user_schema.dump(user)
            return jsonify(result)
        return jsonify({"message": "User not found"}), 404
    except Exception as e:
        app.logger.error("Error fetching user: %s", str(e))
        return jsonify({"message": "An error occurred while fetching the user"}), 500

@app.route('/user', methods=['POST'])
@csrf.exempt  # Example; in practice, configure CSRF protection properly
def add_user():
    data = request.get_json()
    if not data:
        abort(400, description="No input data provided")
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    hashed_password = generate_password_hash(data['password'], method='pbkdf2:sha256')
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully"}), 201

@app.route('/user/<int:id>', methods=['PUT'])
@csrf.exempt
def update_user(id):
    data = request.get_json()
    if not data:
        abort(400, description="No input data provided")
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    try:
        user = User.query.get(id)
        if user:
            user.username = data['username']
            user.password = generate_password_hash(data['password'], method='pbkdf2:sha256')
            db.session.commit()
            return jsonify({"message": "User updated successfully"})
        return jsonify({"message": "User not found"}), 404
    except Exception as e:
        app.logger.error("Error updating user: %s", str(e))
        return jsonify({"message": "An error occurred while updating the user"}), 500

@app.route('/user/<int:id>', methods=['DELETE'])
@csrf.exempt
def delete_user(id):
    try:
        user = User.query.get(id)
        if user:
            db.session.delete(user)
            db.session.commit()
            return jsonify({"message": "User deleted successfully"})
        return jsonify({"message": "User not found"}), 404
    except Exception as e:
        app.logger.error("Error deleting user: %s", str(e))
        return jsonify({"message": "An error occurred while deleting the user"}), 500

# Start the Flask app in a separate thread
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5001}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.18.0.56:5001
INFO:werkzeug:Press CTRL+C to quit


In [19]:
!curl -X POST http://127.0.0.1:5001/user -H "Content-Type: application/json" -d '{"username":"testuser","password":"securepassword"}'

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>


In [ ]:
!curl -X GET http://127.0.0.1:5001/users

In [ ]:
!curl -X GET http://localhost:5001/user/1

In [ ]:
!curl -X PUT http://localhost:5001/user/1 -H "Content-Type: application/json" -d '{"username":"testuser","password":"newsecurepassword"}'

In [ ]:
!curl -X DELETE http://localhost:5000/user/1